In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

trans = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/newdata.csv')

In [ ]:
trans.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15349956 entries, 0 to 15349955
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   chain             int64  
 2   dept              int64  
 3   category          int64  
 4   company           int64  
 5   brand             int64  
 6   date              object 
 7   productsize       float64
 8   productmeasure    object 
 9   purchasequantity  int64  
 10  purchaseamount    float64
dtypes: float64(2), int64(7), object(2)
memory usage: 2.8 GB


In [ ]:
for dtype in ['float','int','object']:
    selected_dtype = trans.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 78.07 MB
Average memory usage for int columns: 102.47 MB
Average memory usage for object columns: 614.83 MB


In [ ]:
import numpy as np
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)
trans_int = trans.select_dtypes(include=['int'])
converted_int = trans_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(trans_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([trans_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

819.78 MB
483.08 MB


,before,after
uint8,NaN,1
uint16,NaN,2
uint32,NaN,1
int64,7.0,1
uint64,NaN,2


In [ ]:
trans_float = trans.select_dtypes(include=['float'])
converted_float = trans_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(trans_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([trans_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

234.22 MB
117.11 MB


,before,after
float32,NaN,2.0
float64,2.0,NaN


In [ ]:
optimized_trans = trans.copy()
optimized_trans[converted_int.columns] = converted_int
optimized_trans[converted_float.columns] = converted_float
print(mem_usage(trans))
print(mem_usage(optimized_trans))

2898.49 MB
2444.69 MB


Object

In [ ]:
trans_obj = trans.select_dtypes(include=['object']).copy()
trans_obj.describe()

,date,productmeasure
count,15349956,15349956
unique,514,3
top,2013-02-02,OZ
freq,56000,14875161


In [ ]:
dow = trans_obj.date
print(dow.head())
dow_cat = dow.astype('category')
print(dow_cat.head())

0    2012-03-02
1    2012-03-02
2    2012-03-06
3    2012-03-07
4    2012-03-14
Name: date, dtype: object
0    2012-03-02
1    2012-03-02
2    2012-03-06
3    2012-03-07
4    2012-03-14
Name: date, dtype: category
Categories (514, object): ['2012-03-02', '2012-03-03', '2012-03-04', '2012-03-05', ..., '2013-07-25',
                           '2013-07-26', '2013-07-27', '2013-07-28']


In [ ]:
dow_cat.head().cat.codes

0     0
1     0
2     4
3     5
4    12
dtype: int16

In [ ]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

980.80 MB
29.33 MB


In [ ]:
converted_obj = pd.DataFrame()
for col in trans_obj.columns:
    num_unique_values = len(trans_obj[col].unique())
    num_total_values = len(trans_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = trans_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = trans_obj[col]

In [ ]:
print(mem_usage(trans_obj))
print(mem_usage(converted_obj))
compare_obj = pd.concat([trans_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

1844.50 MB
43.97 MB


,before,after
object,2.0,NaN
category,NaN,1.0
category,NaN,1.0


In [ ]:
optimized_trans[converted_obj.columns] = converted_obj
mem_usage(optimized_trans)

'644.16 MB'

In [ ]:
date = optimized_trans.date
print(mem_usage(date))
date.head()

29.33 MB


0    2012-03-02
1    2012-03-02
2    2012-03-06
3    2012-03-07
4    2012-03-14
Name: date, dtype: category
Categories (514, object): ['2012-03-02', '2012-03-03', '2012-03-04', '2012-03-05', ..., '2013-07-25',
                           '2013-07-26', '2013-07-27', '2013-07-28']

In [ ]:
optimized_trans['date'] = pd.to_datetime(date, format = '%Y-%m-%d')
print(mem_usage(optimized_trans))
optimized_trans.date.head()

644.13 MB


0   2012-03-02
1   2012-03-02
2   2012-03-06
3   2012-03-07
4   2012-03-14
Name: date, dtype: category
Categories (514, datetime64[ns]): [2012-03-02, 2012-03-03, 2012-03-04, 2012-03-05, ..., 2013-07-25,
                                   2013-07-26, 2013-07-27, 2013-07-28]

In [ ]:
# pip install pprintpp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dtypes = optimized_trans.drop('date',axis=1).dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))

preview = first2pairs = { key:value for key,value in list(column_types.items())[:10] }
preview

{'id': 'uint64',
 'chain': 'uint16',
 'dept': 'uint8',
 'category': 'uint16',
 'company': 'uint64',
 'brand': 'uint32',
 'productsize': 'float32',
 'productmeasure': 'category',
 'purchasequantity': 'int64',
 'purchaseamount': 'float32'}